# Portfolio Chatbot using Gemini | Gen AI Capstone (2025)

This project showcases a personalized AI chatbot built using Gemini Pro APIs that demonstrates several key Generative AI capabilities:

* **RAG (Retrieval-Augmented Generation):** Answers portfolio-specific questions using information retrieved from a vector database.
* **Function Calling:** Performs real actions by invoking predefined Python functions based on user requests.
* **Embedding Classification with Keras:** Classifies user intent to provide more relevant and targeted responses.

This chatbot utilizes tools like **ChromaDB** for vector search, **Gemini Pro** for content generation and understanding, and **Keras** for embedding classification.

🔗 Built for the [Google Generative AI Intensive Capstone 2025Q1](https://kaggle.com/competitions/gen-ai-intensive-course-capstone-2025q1)

## Install Dependencies

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
!pip install -q chromadb google-generativeai keras sentence-transformers
!pip install -U -q "google-genai==1.7.0"
!pip install "jupyterlab>=3.1.0,<4.0.0a0"
!pip install google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 70.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 86.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Gemini API Setup

In [2]:
# Import libraries
from google import genai
from google.genai import types
from kaggle_secrets import UserSecretsClient
from IPython.display import display, Markdown, HTML
from google.api_core import retry

# Load API Key and Initialize Gemini Client
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})
genai.models.Models.generate_content = retry.Retry(predicate=is_retriable)(genai.models.Models.generate_content)
client = genai.Client(api_key=GOOGLE_API_KEY)

## Basic Gemini Prompt Test

In [3]:
# Test Basic Gemini Prompt
prompt = "Describe telangana City in a sentence..."
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt)
print(response.text)

Telangana is an Indian city known for its rich history, vibrant culture, and rapidly growing IT and pharmaceutical industries, with Hyderabad as its capital.



## Setup ChromaDB for RAG

This section sets up ChromaDB, an open-source vector database, to store embeddings of information about my portfolio. This will enable the chatbot to retrieve relevant context when answering user questions (Retrieval-Augmented Generation - RAG).

We will create a document containing information about my skills, projects, and experiences, generate embeddings for it using Gemini's embedding model, and store it in ChromaDB.

In [4]:
DOCUMENT = """
About Chandu Chitteti
A highly motivated and results-oriented Computer Science Engineering student at NBKRIST with a strong passion for full-stack development and software engineering principles. Possessing hands-on experience across a diverse range of technologies including Django, Reactjs, Flask, SQL, Java, and Python, Chandu is adept at building robust and scalable applications.
Driven by a keen interest in emerging technologies, Chandu is actively exploring the intersection of Artificial Intelligence/Machine Learning and the evolving landscape of Web3. This includes a dedicated effort to understand the workflows and technical intricacies related to cryptocurrency, coins, tokens, and blockchain networks. Chandu is eager to leverage his technical skills and growing knowledge in these areas to contribute to innovative and impactful projects.
---TECHNICAL SKILLS 
Chandu has hands-on experience with technologies like Django, React, SQL, and Java.
---PROJECTS
Intelligent Virtual Personal Assistant (Python): Developed a functional personal assistant using Python, capable of launching local applications, conducting web searches, and even playing YouTube videos based on user commands. This project showcases strong foundational programming skills and the ability to create practical desktop applications.
AI-Assisted Todo Application (Dart/Flutter): Explored the Dart programming language and Flutter framework by developing a todo application, leveraging AI concepts during the development process. This highlights a proactive approach to learning new languages and frameworks.
Real-time E-Bidding Platform (College Laboratory): Designed and implemented an e-bidding system for college laboratory items. The platform facilitates bidding, automatically determines the highest bidder upon timer expiry, and generates winner reports, demonstrating skills in platform development and algorithmic thinking.
Web Development Fundamentals: Proficient in front-end technologies with the creation of a responsive Temperature Converter (HTML, CSS, JavaScript).
Desktop Application Development (Python): Developed several practical desktop applications including a Phonebook with a Graphical User Interface (GUI), a Todo List application, and a YouTube Audio/Video Downloader. These projects demonstrate experience with Python GUI libraries and practical application development.
Custom In-App Keyboard (Python & Reactjs): Designed and implemented a fully functional in-app keyboard as both a standalone Python application and as a reusable component in Reactjs. This highlights versatility in both front-end and back-end development and UI/UX considerations.
Personalized Chatbot (Google Workshop): Recently developed a chatbot during a Google workshop designed to answer user queries specifically about Chandu. This demonstrates an ability to quickly learn and apply new technologies in the domain of conversational AI.
Web3 Exploration: Currently engaged in understanding and developing within the Web3 ecosystem, focusing on the workflows and technologies associated with cryptocurrencies, coins, tokens, and blockchain networks. This reflects a commitment to staying at the forefront of technological advancements.
Chandu's diverse project experience showcases a strong aptitude for problem-solving, a dedication to continuous learning, and the ability to translate theoretical knowledge into practical and functional applications across various technology stacks. His current exploration into Web3 further underscores his ambition to contribute to the next generation of the internet.
---INTERNSHIPS AND WORK EXPERIENCE
Chandu has completed internships with Oasis Infobyte and CodSoft, where he worked on web development and Python GUI applications. At CompSci Technology Solutions, he was promoted from Intern to Intern Lead, demonstrating strong leadership and technical abilities.
---ACHIEVEMENTS AND CERTIFICATIONS
He’s ranked in the top 1k nationally(704) in TCS CodeVita and has obtained certifications such as NPTEL’s Ethical Hacking, where he explored tools like airmon-ng and Crunch.
---CODING STYLE AND PREFERENCES
He prefers coding in a clean and concise style, avoiding unnecessary comments, and structures code for clarity and performance. Chandu primarily develops on Linux and is comfortable with workflow automation and system-level programming.
---FUTURE GOALS
Driven to transition into the IT industry and secure a rewarding role that facilitates a deep understanding of professional workflows and industry best practices. Eager to collaborate with experienced professionals within dynamic development organizations, Chandu is committed to continuous personal and professional growth. He aims to actively contribute to team success while expanding his skillset and industry knowledge, fostering both technical expertise and interpersonal development.
---LEARNING PHILOSOPHY
Chandu believes in the 25-75 rule for learning (25% theory, 75% practice) and uses a blend of AI tools and manual processes to optimize his workflow.
---COMPETITIVE PROGRAMMING
Chandu is a competitive programmer with solid skills in DSA, problem-solving, and algorithm design.
---CURRENT ACTIVITIES
After workign on Ai-Ml for quite a tile, now he is actively working to understand Web3 workflows and technologies related to cryptocurrency, coins, tokens, and networks. This indicates he is in a learning and exploratory phase in this domain.
"""

documents = [doc.strip() for doc in DOCUMENT.split('---') if doc.strip()]

print(f"Number of documents: {len(documents)}")
for i, doc in enumerate(documents):
    preview = doc[:90] + "..." if len(doc) > 30 else doc
    print(f"DOCUMENT{i+1} = \"\"\"{preview}\"\"\"")

Number of documents: 10
DOCUMENT1 = """About Chandu Chitteti
A highly motivated and results-oriented Computer Science Engineering..."""
DOCUMENT2 = """TECHNICAL SKILLS 
Chandu has hands-on experience with technologies like Django, React, SQL..."""
DOCUMENT3 = """PROJECTS
Intelligent Virtual Personal Assistant (Python): Developed a functional personal ..."""
DOCUMENT4 = """INTERNSHIPS AND WORK EXPERIENCE
Chandu has completed internships with Oasis Infobyte and C..."""
DOCUMENT5 = """ACHIEVEMENTS AND CERTIFICATIONS
He’s ranked in the top 1k nationally(704) in TCS CodeVita ..."""
DOCUMENT6 = """CODING STYLE AND PREFERENCES
He prefers coding in a clean and concise style, avoiding unne..."""
DOCUMENT7 = """FUTURE GOALS
Driven to transition into the IT industry and secure a rewarding role that fa..."""
DOCUMENT8 = """LEARNING PHILOSOPHY
Chandu believes in the 25-75 rule for learning (25% theory, 75% practi..."""
DOCUMENT9 = """COMPETITIVE PROGRAMMING
Chandu is a competitive programmer with s

## Creating Embedding Function

In [5]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry
from google.genai import types

# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents or queries
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

## Initialize ChromaDB and Add Document 

In [6]:
import chromadb

DB_NAME = "portfolio_db"
embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True
chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

# Add the split documents to ChromaDB with unique IDs
ids = [f"doc_{i+1}" for i in range(len(documents))]
db.add(documents=documents, ids=ids)

print(f"Number of items in ChromaDB: {db.count()}")
print(db.peek(3)) # Peek at a few of the inserted documents

/tmp/ipykernel_31/4281459235.py:4: DeprecationWarning: The class GeminiEmbeddingFunction does not implement __init__. This will be required in a future version.
  embed_fn = GeminiEmbeddingFunction()


Number of items in ChromaDB: 10
{'ids': ['doc_1', 'doc_2', 'doc_3'], 'embeddings': array([[ 0.0322766 , -0.03686123, -0.06438102, ..., -0.03469459,
        -0.01017373,  0.00315272],
       [ 0.02607657, -0.05078001, -0.07925962, ..., -0.02605872,
         0.01231599, -0.01583621],
       [ 0.01498265, -0.04928812, -0.07429814, ..., -0.01349979,
         0.02035135,  0.01084798]]), 'documents': ['About Chandu Chitteti\nA highly motivated and results-oriented Computer Science Engineering student at NBKRIST with a strong passion for full-stack development and software engineering principles. Possessing hands-on experience across a diverse range of technologies including Django, Reactjs, Flask, SQL, Java, and Python, Chandu is adept at building robust and scalable applications.\nDriven by a keen interest in emerging technologies, Chandu is actively exploring the intersection of Artificial Intelligence/Machine Learning and the evolving landscape of Web3. This includes a dedicated effort to

## Retrieval: Find relevant documents, Test RAG with a Query

In [7]:
# Switch to query mode for embedding the query
embed_fn.document_mode = False

# Define a question that should be answered by one of the specific sections
query = "Tell me about his completed projects"

# Search the Chroma DB using the query
result = db.query(query_texts=[query], n_results=1)
relevant_document = result["documents"][0] if result["documents"] else "No relevant information found."

print("Retrieved Relevant Document:")
print(relevant_document)

Retrieved Relevant Document:
["PROJECTS\nIntelligent Virtual Personal Assistant (Python): Developed a functional personal assistant using Python, capable of launching local applications, conducting web searches, and even playing YouTube videos based on user commands. This project showcases strong foundational programming skills and the ability to create practical desktop applications.\nAI-Assisted Todo Application (Dart/Flutter): Explored the Dart programming language and Flutter framework by developing a todo application, leveraging AI concepts during the development process. This highlights a proactive approach to learning new languages and frameworks.\nReal-time E-Bidding Platform (College Laboratory): Designed and implemented an e-bidding system for college laboratory items. The platform facilitates bidding, automatically determines the highest bidder upon timer expiry, and generates winner reports, demonstrating skills in platform development and algorithmic thinking.\nWeb Develop

In [8]:
# Include the retrieved document in the prompt to Gemini
prompt = f"""Answer the following question based on the provided document:

Question: {query}

Document: {relevant_document}
"""
# Generate the answer using the correct Gemini Pro model name
response = client.models.generate_content(
    model="gemini-2.0-flash",  # Replace with the correct model name
    contents=prompt)

Markdown(response.text)

Based on the document, here are Chandu's completed projects:

*   **Intelligent Virtual Personal Assistant (Python):** Launches applications, conducts web searches, and plays YouTube videos.
*   **AI-Assisted Todo Application (Dart/Flutter):** A todo application using Dart/Flutter, incorporating AI concepts.
*   **Real-time E-Bidding Platform (College Laboratory):** An e-bidding system for college laboratory items.
*   **Responsive Temperature Converter (HTML, CSS, JavaScript):** A web-based temperature converter.
*   **Desktop Applications (Python):** Phonebook with GUI, Todo List application, and YouTube Audio/Video Downloader.
*   **Custom In-App Keyboard (Python & Reactjs):** Implemented as both a standalone Python application and as a reusable component in Reactjs.
*   **Personalized Chatbot (Google Workshop):** A chatbot designed to answer user queries about Chandu.


## Embedding Classification for Intent Recognition

This section demonstrates how to classify user queries based on their embeddings using a Keras model. This allows the chatbot to understand the user's intent and potentially trigger different actions or provide more tailored responses.

We will create a small dataset of example user queries and their corresponding intents, generate embeddings for these queries, train a simple Keras classification model, and then use this model to predict the intent of new user queries.

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from sentence_transformers import SentenceTransformer

intent_data = {
    # Projects
    "What projects have you worked on?": "projects",
    "Tell me about your projects.": "projects",
    "Can you list your projects?": "projects",
    "What are some of the projects you've done?": "projects",
    "Describe the projects you've been involved in.": "projects",
    "Talk about your past projects.": "projects",
    "What kind of projects do you do?": "projects",
    "Have you built any applications?": "projects",
    "Software projects you've completed?": "projects",
    "Details about your development projects.": "projects",

    # Skills
    "Tell me about your skills.": "skills",
    "What are your technical skills?": "skills",
    "What skills do you possess?": "skills",
    "Can you list your abilities?": "skills",
    "What technologies are you proficient in?": "skills",
    "Your areas of expertise?": "skills",
    "What programming languages do you know?": "skills",
    "What frameworks are you familiar with?": "skills",
    "Skills you have acquired?": "skills",
    "Your technical proficiencies?": "skills",

    # Contact
    "How can I contact you?": "contact",
    "What is your contact information?": "contact",
    "How do I get in touch with you?": "contact",
    "What's your email address?": "contact",
    "Do you have a phone number?": "contact",
    "Where can I reach you?": "contact",
    "Ways to connect with you?": "contact",
    "Let me know your contact details.": "contact",
    "How to reach out?": "contact",
    "Your preferred method of contact?": "contact",

    # Education
    "What is your educational background?": "education",
    "Tell me about your education.": "education",
    "What degrees do you have?": "education",
    "Where did you study?": "education",
    "Your academic qualifications?": "education",
    "What did you study at NBKRIST?": "education",
    "Your college information?": "education",
    "Details about your schooling.": "education",
    "Your educational history?": "education",
    "What courses have you taken?": "education",

    # Learning
    "What are you currently learning?": "learning",
    "What are you studying now?": "learning",
    "What new things are you exploring?": "learning",
    "What technologies are you currently focused on?": "learning",
    "What areas are you developing your knowledge in?": "learning",
    "What's next on your learning list?": "learning",
    "Things you are trying to understand?": "learning",
    "Your current learning interests?": "learning",
    "What are you working to understand?": "learning",
    "New skills you are acquiring?": "learning",

    # Experience
    "Describe your work experience.": "experience",
    "Tell me about your professional experience.": "experience",
    "What kind of work have you done?": "experience",
    "Where have you worked before?": "experience",
    "Your employment history?": "experience",
    "Talk about your internships.": "experience",
    "Your past roles?": "experience",
    "Companies you have worked with?": "experience",
    "Your professional background?": "experience",
    "Details of your work history.": "experience",

    # Achievements
    "What are your achievements?": "achievements",
    "Tell me about your accomplishments.": "achievements",
    "What have you achieved?": "achievements",
    "Any notable successes?": "achievements",
    "Things you are proud of?": "achievements",
    "Your significant accomplishments?": "achievements",
    "Have you received any awards?": "achievements",
    "Your rankings or recognitions?": "achievements",
    "Things you have excelled at?": "achievements",
    "Your key achievements so far?": "achievements"
}

queries = list(intent_data.keys())
labels = list(intent_data.values())

# Use a pre-trained Sentence Transformer model for generating embeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
query_embeddings = embedding_model.encode(queries)

# Convert labels to numerical values
unique_labels = list(set(labels))
label_to_index = {label: i for i, label in enumerate(unique_labels)}
numerical_labels = np.array([label_to_index[label] for label in labels])

# Split data into training and testing sets
train_embeddings, test_embeddings, train_labels, test_labels = train_test_split(
    query_embeddings, numerical_labels, test_size=0.2, random_state=42
)

# Inspect Labels (Added for debugging)
print("Unique Labels:", unique_labels)
print("Label to Index Mapping:", label_to_index)
print("Numerical Labels:", numerical_labels)
print("Train Labels:", train_labels)
print("Test Labels:", test_labels)

# Define the Keras model using the Functional API
num_classes = len(unique_labels)
embedding_dimension = train_embeddings.shape[1]

input_layer = keras.Input(shape=(embedding_dimension,), name="input_embedding")
dense_1 = layers.Dense(128, activation="relu", name="dense_1")(input_layer)
output_layer = layers.Dense(num_classes, activation="softmax", name="output")(dense_1)

classification_model = keras.Model(inputs=input_layer, outputs=output_layer, name="intent_classifier")

# Compile the model
classification_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
classification_model.fit(train_embeddings, train_labels, epochs=10, verbose=0)

# Evaluate the model
loss, accuracy = classification_model.evaluate(test_embeddings, test_labels, verbose=0)
print(f"Embedding Classification Model Accuracy: {accuracy:.2f}")

2025-04-13 13:10:21.280969: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744549821.538559      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744549821.616490      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Unique Labels: ['contact', 'education', 'experience', 'achievements', 'projects', 'skills', 'learning']
Label to Index Mapping: {'contact': 0, 'education': 1, 'experience': 2, 'achievements': 3, 'projects': 4, 'skills': 5, 'learning': 6}
Numerical Labels: [4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 6 6 6 6 6 6 6 6 6 6 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3]
Train Labels: [3 1 2 1 2 6 5 1 6 0 4 2 6 6 6 5 2 1 0 1 5 2 4 5 4 2 4 1 3 3 3 5 0 6 0 6 0
 2 3 5 1 3 3 1 0 6 3 4 2 0 4 0 0 3 5 2]
Test Labels: [0 4 6 4 2 5 5 1 6 5 1 4 3 4]


2025-04-13 13:11:01.420935: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Embedding Classification Model Accuracy: 0.21


## Function Calling for Real Actions

This section demonstrates the use of Gemini's function calling capability. We will define Python functions that the chatbot can invoke to perform specific actions related to the portfolio. When a user query indicates the need for such an action, Gemini can generate a structured function call, which our code will then execute, and the result will be used to generate a natural language response back to the user.

Here, we will define a few example functions: `get_educational_background` and `get_contact_information`.

In [10]:
def get_contact_information():
    contact_info = {
        "Email": "chanduchitteti@outlook.com",
        "LinkedIn": "[https://www.linkedin.com/in/chandu-chitteti-...](https://www.linkedin.com/in/chandu-chitteti-...)",
        "Portfolio": "[https://chanduchitteti.netlify.app/](https://chanduchitteti.netlify.app/)"
    }
    markdown_output = "**Contact Information:**\n\n"
    for key, value in contact_info.items():
        markdown_output += f"- **{key}:** {value}\n"
    return Markdown(markdown_output)

def get_educational_background():
    education_info = {
        "Degree": "Bachelor of Technology in CSE",
        "University": "JNTUA",
        "College": "NBKRIST",
        "Graduation": "2026"
    }
    markdown_output = "**Educational Background:**\n\n"
    for key, value in education_info.items():
        markdown_output += f"- **{key}:** {value}\n"
    return Markdown(markdown_output)

## Obtaining user intent for user query and then obtaining results for user queries.

In [19]:
from google.generativeai import GenerativeModel
from IPython.display import Markdown, display
import google.generativeai as genai
import json
import regex

# Initialize Gemini Client again after loading the key (explicit)
genai.configure(api_key=GOOGLE_API_KEY)

# Initialize Gemini models
intent_model = genai.GenerativeModel(model_name="gemini-2.0-flash", generation_config={"temperature": 0.3})
response_model = genai.GenerativeModel(model_name="gemini-pro")

def Rag_Query_handler(query):
    embed_fn.document_mode = False

    # Define a question that should be answered by one of the specific sections
    rag_query = "Tell me about his completed projects" # Use a consistent variable name
    print(f"RAG Query: {rag_query}")

    # Search the Chroma DB using the query
    result = db.query(query_texts=[rag_query], n_results=1)
    relevant_document = result["documents"][0] if result["documents"] else "No relevant information found."
    # Include the retrieved document in the prompt to Gemini
    prompt = f"""Answer the following question based on the provided document:

    Question: {query}

    Document: {relevant_document}
    """
    # Generate the answer using the correct Gemini Pro model name
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",  # Replace with the correct model name
            contents=prompt)
        return Markdown(response.text)
    except Exception as e:
        print(f"Error during Gemini RAG response: {e}")
        return Markdown("Error generating response from retrieved document.")

def handle_user_query(query):
    print(f"User Query: {query}")
    intent_prompt = f"""Identify the intent of the following user query. Choose ONE of the following precise categories:
    Education, contact, Outerscope, project_details, list_projects, skills, experience, achievements, general_info, greeting, unrelated.
    Return only the single category.

    User Query: {query}
    Intent: """
    try:
        intent_response = intent_model.generate_content(contents=intent_prompt)
        userintent = intent_response.text.strip()
        print(f"Predicted Intent: {userintent}\n")
    except Exception as e:
        print(f"Error during Gemini intent classification: {e}")
        return Markdown("Error understanding your request.")

    if userintent.lower() == "education":
        return get_educational_background()
    elif userintent.lower() == "contact":
        return get_contact_information()
    elif userintent.lower() == "outerscope" or userintent.lower() == "unrelated":
        prompt = f"Respond to the user query: {query}"
        try:
            response = response_model.generate_content(contents=prompt)
            return Markdown(response.text)
        except Exception as e:
            print(f"Error during Gemini outerscope response: {e}")
            return Markdown("I'm not sure how to answer that.")
    else:
        print("User intent is:", userintent)
        return Rag_Query_handler(query)

queries = [
    "Tell me about his education",
    "What is his contact information?",
    "What is the capital of France?",
    "Tell me about his completed projects",
    "What are his skills?"
]

for query in queries:
    response = handle_user_query(query)
    print(f"\n---------------Query: '{query}'---------------")
    display(response)

User Query: Tell me about his education
Predicted Intent: Education


---------------Query: 'Tell me about his education'---------------


**Educational Background:**

- **Degree:** Bachelor of Technology in CSE
- **University:** JNTUA
- **College:** NBKRIST
- **Graduation:** 2026


User Query: What is his contact information?
Predicted Intent: contact


---------------Query: 'What is his contact information?'---------------


**Contact Information:**

- **Email:** chanduchitteti@outlook.com
- **LinkedIn:** [https://www.linkedin.com/in/chandu-chitteti-...](https://www.linkedin.com/in/chandu-chitteti-...)
- **Portfolio:** [https://chanduchitteti.netlify.app/](https://chanduchitteti.netlify.app/)


User Query: What is the capital of France?
Predicted Intent: general_info

User intent is: general_info
RAG Query: Tell me about his completed projects

---------------Query: 'What is the capital of France?'---------------


The document does not contain the answer to the question "What is the capital of France?".


User Query: Tell me about his completed projects
Predicted Intent: project_details

User intent is: project_details
RAG Query: Tell me about his completed projects

---------------Query: 'Tell me about his completed projects'---------------


Here's a summary of his completed projects, based on the provided document:

*   **Intelligent Virtual Personal Assistant (Python):** A personal assistant capable of launching applications, conducting web searches, and playing YouTube videos.
*   **AI-Assisted Todo Application (Dart/Flutter):** A todo application that incorporates AI concepts.
*   **Real-time E-Bidding Platform (College Laboratory):** An e-bidding system for college laboratory items, including bidding, winner determination, and report generation.
*   **Responsive Temperature Converter (HTML, CSS, JavaScript):** A front-end web application.
*   **Desktop Applications (Python):** Several applications, including a Phonebook with GUI, a Todo List, and a YouTube Audio/Video Downloader.
*   **Custom In-App Keyboard (Python & Reactjs):** A keyboard implemented as a standalone Python application and as a reusable Reactjs component.
*   **Personalized Chatbot (Google Workshop):** A chatbot designed to answer user queries about Chandu.


User Query: What are his skills?

---------------Query: 'What are his skills?'---------------


Based on the provided document, his skills include:

*   **Programming:** Python, Dart, HTML, CSS, JavaScript, Reactjs
*   **Frameworks/Libraries:** Flutter, Python GUI libraries
*   **Application Development:** Desktop applications, web applications, mobile applications (with Flutter)
*   **AI/ML:** AI concepts, Conversational AI (Chatbot development)
*   **Web Development:** Front-end development, responsive design
*   **Platform Development:** E-bidding platform design and implementation
*   **UI/UX:** Designing and implementing UI components like a custom in-app keyboard
*   **Algorithmic Thinking:** Demonstrated through the e-bidding platform project
*   **Problem-solving:** Evidenced by his diverse projects
*   **Continuous Learning:** Proactive approach to learning new languages and frameworks, and exploring Web3 technologies.
*   **Web3:** Understanding and developing within the Web3 ecosystem.


# Queries without having Function calls

In [20]:
from IPython.display import display

print("---------------General Query 1---------------")
general_query_1 = "what is chandu's educational qualifications?"
display(handle_user_query(general_query_1))

print("---------------General Query 2---------------")
general_query_2 = "Tell me about chandu's experience with web development."
display(handle_user_query(general_query_2))

print("---------------General Query 3---------------")
general_query_3 = "What kind of projects are chandu interested in?"
display(handle_user_query(general_query_3))

---------------General Query 1---------------
User Query: what is chandu's educational qualifications?
Predicted Intent: Education



**Educational Background:**

- **Degree:** Bachelor of Technology in CSE
- **University:** JNTUA
- **College:** NBKRIST
- **Graduation:** 2026


---------------General Query 2---------------
User Query: Tell me about chandu's experience with web development.
Predicted Intent: experience

User intent is: experience
RAG Query: Tell me about his completed projects


Based on the document, Chandu has experience with web development in the following ways:

*   **Web Development Fundamentals:** He is proficient in front-end technologies using HTML, CSS, and JavaScript, demonstrated by the creation of a responsive Temperature Converter.
*   **Custom In-App Keyboard:** He implemented a fully functional in-app keyboard as a reusable component in Reactjs, showing front-end development skills and UI/UX considerations.
*   **Web3 Exploration:** He is currently learning and developing within the Web3 ecosystem, focusing on related workflows and technologies.
*   **Real-time E-Bidding Platform:** The description lists this project as being done in a "College Laboratory" with no listed technologies, so it can be inferred that web technologies were used for this.


---------------General Query 3---------------
User Query: What kind of projects are chandu interested in?
Predicted Intent: list_projects

User intent is: list_projects
RAG Query: Tell me about his completed projects


Based on the document, Chandu is interested in projects involving:

*   **Intelligent Virtual Personal Assistants:** Developing functional personal assistants using Python.
*   **AI-Assisted Applications:** Integrating AI concepts into application development, such as a todo application using Dart/Flutter.
*   **Platform Development:** Designing and implementing platforms like e-bidding systems.
*   **Web Development:** Creating responsive web applications using HTML, CSS, and JavaScript.
*   **Desktop Application Development:** Building practical desktop applications using Python.
*   **UI/UX Design:** Developing custom in-app keyboards using Python and Reactjs.
*   **Conversational AI:** Building chatbots, particularly using technologies from Google workshops.
*   **Web3 Ecosystem:** Understanding and developing within the Web3 ecosystem, focusing on cryptocurrencies and blockchain technologies.


## Final Notes

✅ **Capstone Capabilities Demonstrated:**

1.  **Retrieval Augmented Generation (RAG):** The chatbot answers general portfolio-related questions by retrieving relevant information from the ChromaDB vector store, demonstrating RAG. See the sections "Setup ChromaDB for RAG" and the query examples.

2.  **Function Calling:** The chatbot utilizes function calling to perform specific actions based on user requests. Python functions (`get_educational_background`, `get_contact_information`) are defined and described to the Gemini model, allowing it to generate structured function calls that are then executed. This is demonstrated in the "Function Calling Setup" and "Chatbot Logic with Function Calling" sections.

3.  **Embedding Classification with Keras:** User queries are classified based on their embeddings using a Keras model to understand the user's intent. This classification helps in potentially routing the query to the appropriate response mechanism or function. The implementation is shown in the "Embedding Classification Setup" section.

This enhanced chatbot architecture effectively integrates Gemini Pro for natural language understanding and generation, ChromaDB for efficient information retrieval, and Keras for intent classification, showcasing a more sophisticated and capable portfolio assistant. This can be further extended with more functions, a larger knowledge base, and a more interactive user interface.